# Question Answering Model 
## no trainer

- dataset
- torch
- transformers
- transformers[torch]
- evaluate

import packages

In [70]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    default_data_collator,
    get_scheduler,
    AutoModelForQuestionAnswering,
)
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch

import evaluate
import collections
from tqdm.auto import tqdm
import numpy as np

import os
import re
import datetime

Set cache directory.

In [71]:
model_dir = '/mount/arbeitsdaten31/studenten1/linku/models'
CACHE_DIR='/mount/arbeitsdaten31/studenten1/linku/cache'
%set_env TRANSFORMERS_CACHE $CACHE_DIR
%set_env HF_MODULES_CACHE $CACHE_DIR
%set_env HF_DATASETS_CACHE $CACHE_DIR

env: TRANSFORMERS_CACHE=/mount/arbeitsdaten31/studenten1/linku/cache
env: HF_MODULES_CACHE=/mount/arbeitsdaten31/studenten1/linku/cache
env: HF_DATASETS_CACHE=/mount/arbeitsdaten31/studenten1/linku/cache


### arguments.py

args_input.

In [72]:
args_input_ALstrategy = 'MarginSampling'
args_input_initseed = 100 # 1000
args_input_quota = 100 # 1000
args_input_batch = 35 # 128
args_input_dataset_name = 'SQuAD'
args_input_iteration = 1
args_input_model_batch = 8 # already add in arguments.py

### load dataset

In [73]:
squad = load_dataset(args_input_dataset_name.lower())
# squad["train"] = squad["train"].shuffle(42).select(range(2000))
squad["train"] = squad["train"].select(range(3000))
squad["validation"] = squad["validation"].select(range(1000))

Found cached dataset squad (/home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 99.76it/s]


Next we will preprocess the dataset (training and evaluation data).

In [74]:
def preprocess_training_features(examples):
    # keep ["offset_mapping"], for compute_metrics()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["id"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [75]:
def preprocess_training_examples(examples):
    # no ['offset_mapping'], for .train() and .eval()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["id"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [76]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [77]:
max_length = 384
stride = 128

# load tokenizer for dataset preprocessing
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# preprocess data
train_dataset = squad["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=squad["train"].column_names,
)
train_features = squad["train"].map(
    preprocess_training_features,
    batched=True,
    remove_columns=squad["train"].column_names,
)
val_dataset = squad["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad["validation"].column_names,
)
val_features = squad["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad["validation"].column_names,
)

Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-d10ddabde563b597.arrow
Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-773e51dc4558cf67.arrow
Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-0f087a92e258c27b.arrow
Loading cached processed dataset at /home/users1/linku/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-0f087a92e258c27b.arrow


In [78]:
train_dataset.set_format("torch")
train_features.set_format("torch")
val_dataset = val_dataset.remove_columns(["offset_mapping"])
val_dataset.set_format("torch")
val_features.set_format("torch")

## model.py

In [79]:
def to_train(num_train_epochs, train_dataloader, device, model, optimizer, lr_scheduler):
	print('num of train dataset', len(train_dataloader.dataset))
	for epoch in range(num_train_epochs):
		model.train()
		for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
			batch = {key: value.to(device) for key, value in batch.items()}
			outputs = model(**batch)
			loss = outputs.loss
			loss.backward()

			optimizer.step()
			lr_scheduler.step()
			optimizer.zero_grad()
    
		model_to_save = model.module if hasattr(model, 'module') else model
		model_to_save.save_pretrained(model_dir)
	print('TRAIN done!')

In [80]:
metric = evaluate.load("squad")

In [81]:
def compute_metrics(start_logits, end_logits, features, examples):
    
    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples, desc="Computing metrics"):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [82]:
def get_pred(eval_dataloader, device, features, examples):
    model = AutoModelForQuestionAnswering.from_pretrained(model_dir).to(device)
    
    test_loss = []
    model.eval()
    start_logits = []
    end_logits = []
    for batch in tqdm(eval_dataloader, desc="Evaluating_pred"):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            print(outputs)
            print(outputs.loss)
            test_loss.append(outputs.loss)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    # if record_loss:
    #     test_loss /= len(eval_dataloader.dataset)
    print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))

    return compute_metrics(start_logits, end_logits, features, examples)

In [83]:
def get_prob(eval_dataloader, device, features, examples):
    model = AutoModelForQuestionAnswering.from_pretrained(model_dir).to(device)

    model.eval()
    start_logits = []
    end_logits = []

    for batch in tqdm(eval_dataloader, desc="Evaluating_prob"):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    prob_dict = {}
    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20 # TODO: if set n_best as 5, will it effect the time??
    
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    for example in tqdm(examples, desc="Computing metrics"):
        example_id = example["id"]
        # context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answers.append(start_logit[start_index] + end_logit[end_index])
        
            if len(answers) > 1:
                prob_dict[feature_index] = softmax(answers)
            elif example_to_features[example_id] != []:
                prob_dict[feature_index] = np.array([0])
    
    return prob_dict

In [84]:
def get_prob_dropout(eval_dataloader, device, features, examples, n_drop=10):
    # deepAL+: self.clf.train()
    model = AutoModelForQuestionAnswering.from_pretrained(model_dir).to(device)
    model.train()
    # deepAL+: probs = torch.zeros([len(data), len(np.unique(data.Y))])
    # deepAL+: loader = DataLoader(data, shuffle=False, **self.params['loader_te_args'])
    prob_dict = {}
    # deepAL+: for i in range(n_drop):
    # deepAL+:     with torch.no_grad():
    # deepAL+:         for x, y, idxs in loader:
    # deepAL+:             x, y = x.to(self.device), y.to(self.device)
    # deepAL+:             out, e1 = self.clf(x)
    # deepAL+:             prob = F.softmax(out, dim=1)
    # deepAL+:             probs[idxs] += prob.cpu()
    for i in range(n_drop):
        start_logits = []
        end_logits = []
        for batch in tqdm(eval_dataloader, desc="Evaluating_prob_dropout"):
            batch = {key: value.to(device) for key, value in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            start_logits.append(outputs.start_logits.cpu().numpy())
            end_logits.append(outputs.end_logits.cpu().numpy())

        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(features)]
        end_logits = end_logits[: len(features)]

        example_to_features = collections.defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n = 0
        for example in tqdm(examples):
            example_id = example["id"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            if 1 < len(answers) < 200: # pad to same numbers of possible answers
                zero_list = [0] * (200 - len(answers))
                answers.extend(zero_list)
            elif len(answers) >= 200:
                answers = answers[:200]

            if len(answers) > 1:
                if feature_index not in prob_dict:
                    prob_dict[feature_index] = softmax(answers)
                else:
                    prob_dict[feature_index] += softmax(answers)
            elif example_to_features[example_id] != []:
                if feature_index not in prob_dict:
                    prob_dict[feature_index] = np.array([0])   

    for key in prob_dict.keys():
        prob_dict[key] /= n_drop

    return prob_dict

In [85]:
def get_prob_dropout_split(eval_dataloader, device, features, examples, n_drop=10):
    ## use tensor to save the answers
    
    # deepAL+: self.clf.train()
    model = AutoModelForQuestionAnswering.from_pretrained(model_dir).to(device)
    model.train()
    # deepAL+: probs = torch.zeros([len(data), len(np.unique(data.Y))])
    # deepAL+: loader = DataLoader(data, shuffle=False, **self.params['loader_te_args'])
    probs = torch.zeros([n_drop, len(eval_dataloader.dataset), 200])
    for_check = []
    # deepAL+: for i in range(n_drop):
    # deepAL+:     with torch.no_grad():
    # deepAL+:         for x, y, idxs in loader:
    # deepAL+:             x, y = x.to(self.device), y.to(self.device)
    # deepAL+:             out, e1 = self.clf(x)
    # deepAL+:             prob = F.softmax(out, dim=1)
    # deepAL+:             probs[i][idxs] += F.softmax(out, dim=1).cpu()
    for i in range(n_drop):
        prob_dict = {}
        start_logits = []
        end_logits = []
        for batch in tqdm(eval_dataloader, desc="Evaluating_prob_dropout"):
            batch = {key: value.to(device) for key, value in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            start_logits.append(outputs.start_logits.cpu().numpy())
            end_logits.append(outputs.end_logits.cpu().numpy())

        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(features)]
        end_logits = end_logits[: len(features)]

        example_to_features = collections.defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n = 0
        for example in tqdm(examples, desc="Computing metrics"):
            example_id = example["id"]
            # context = example["context"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            
                if 1 < len(answers) < 200: # pad to same numbers of possible answers
                    zero_list = [0] * (200 - len(answers))
                    answers.extend(zero_list)
                elif len(answers) >= 200:
                    answers = answers[:200]

                probs[i][feature_index] += torch.tensor(softmax(answers))

            # if n == 0 and len(softmax(answers)) > 1:
            #     for_check.append(answers[:5])
            #     n += 1 

    # return prob_dict, for_check
    return probs

## utils.py

In [86]:
def get_unlabel_data(n_pool, labeled_idxs, train_dataset):
    unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
    unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
    return unlabeled_idxs, unlabeled_data

In [87]:
def softmax(x): 
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

## Query

In [88]:
def random_sampling_query(labeled_idxs, n):
    return np.random.choice(np.where(labeled_idxs==0)[0], n, replace=False)

In [89]:
def margin_sampling_query(n_pool, labeled_idxs, train_dataset, train_features, examples, model, device, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    # TODO: print for recording
    print('Margin querying starts!')
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_dict = get_prob(unlabeled_dataloader, device, unlabeled_features, examples)
    # TODO: print for recording
    print('Got probability!')
    # deepAL+: probs_sorted, _ = probs.sort(descending=True)
    # deepAL+: uncertainties = probs_sorted[:, 0] - probs_sorted[:,1]
    uncertainties_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            sort_probs = np.sort(probs)[::-1] # This method returns a copy of the array, leaving the original array unchanged.
            uncertainties_dict[idx] = sort_probs[0] - sort_probs[1]
        elif idx:
            uncertainties_dict[idx] = np.array([0])

    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]] 
    sorted_uncertainties_list = sorted(uncertainties_dict.items(), key=lambda x: x[1], reverse=True)
    
    return unlabeled_idxs[[idx for (idx, uncertainties) in sorted_uncertainties_list[:n]]]

In [90]:
def least_confidence_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    # TODO: print for recording
    print('LC querying starts!')
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_dict = get_prob(unlabeled_dataloader, device, unlabeled_features, examples)
    # TODO: print for recording
    print('Got probability!')
    # deepAL+: uncertainties = probs.max(1)[0]
    confidence_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            confidence_dict[idx] = max(probs)
        elif idx:
            confidence_dict[idx] = np.array([0])

    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
    sorted_confidence_list = sorted(confidence_dict.items, key=lambda x: x[1])
    return unlabeled_idxs[[idx for (idx, confidence) in sorted_confidence_list[:n]]]

In [91]:
def var_ratio_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    # TODO: print for recording
    print('Var Ratio querying starts!')
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_dict = get_prob(unlabeled_dataloader, device, unlabeled_features, examples)
    # TODO: print for recording
    print('Got probability!')
    # deepAL+: preds = torch.max(probs, 1)[0]
    # deepAL+: uncertainties = 1.0 - preds
    confidence_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            confidence_dict[idx] = 1.0 - max(probs)
        elif idx:
            confidence_dict[idx] = np.array([0])

    # deepAL+: return unlabeled_idxs[uncertainties.sort(descending=True)[1][:n]]
    sorted_confidence_list = sorted(confidence_dict.items, key=lambda x: x[1], reverse=True)
    return unlabeled_idxs[[idx for (idx, confidence) in sorted_confidence_list[:n]]]
# comment for the same query as LC

In [92]:
def entropy_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    # TODO: print for recording
    print('Entropy querying starts!')
    prob_dict = get_prob(unlabeled_dataloader, device, unlabeled_features, examples)
    # TODO: print for recording
    print('Got probability!')
    # deepAL+: log_probs = torch.log(probs)
    # deepAL+: uncertainties = (probs*log_probs).sum(1)
    entropy_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            log_probs = np.log(probs)
            entropy_dict[idx] = (probs*log_probs).sum()
        elif idx:
            entropy_dict[idx] = np.array([0])
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
    sorted_entropy_list = sorted(entropy_dict.items(), key=lambda x: x[1])
    return unlabeled_idxs[[idx for (idx, entropy) in sorted_entropy_list[:n]]]

In [93]:
def margin_sampling_dropout_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n):
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    # TODO: print for recording
    print('Margin dropout querying starts!')
    prob_dict = get_prob_dropout(unlabeled_dataloader, device, unlabeled_features, examples)
    # TODO: print for recording
    print('Got probability!')
    uncertainties_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            sort_probs = np.sort(probs)[::-1] # This method returns a copy of the array, leaving the original array unchanged.
            uncertainties_dict[idx] = sort_probs[0] - sort_probs[1]
        elif idx:
            uncertainties_dict[idx] = np.array([0])

    sorted_uncertainties_list = sorted(uncertainties_dict.items(), key=lambda x: x[1], reverse=True)
    return unlabeled_idxs[[idx for (idx, uncertainties) in sorted_uncertainties_list[:n]]]

In [94]:
def mean_std_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
  		unlabeled_data,
      shuffle=True,
      collate_fn=default_data_collator,
      batch_size=8,
    )
    # TODO: print for recording
    print('Mean STD querying starts!')
    # deepAL+: probs = self.predict_prob_dropout_split(unlabeled_data, n_drop=self.n_drop).numpy()
    probs = get_prob_dropout_split(unlabeled_dataloader, device, unlabeled_features, examples).numpy()
    # TODO: print for recording
    print('Got probability!')
    # deepAL+: sigma_c = np.std(probs, axis=0)
    sigma_c = np.std(probs, axis=0)
    # deepAL+: uncertainties = torch.from_numpy(np.mean(sigma_c, axis=-1))
    uncertainties = torch.from_numpy(np.mean(sigma_c, axis=-1)) # use tensor.sort() will sort the data and produce sorted indexes
    # deepAL+: return unlabeled_idxs[uncertainties.sort(descending=True)[1][:n]] # [1]: to get sorted data's indexes
    return unlabeled_idxs[uncertainties.sort(descending=True)[1][:n]]

In [95]:
def bayesian_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
      unlabeled_data,
      shuffle=True,
      collate_fn=default_data_collator,
      batch_size=8,
    )
    # deepAL+: probs = self.predict_prob_dropout_split(unlabeled_data, n_drop=self.n_drop)
    probs = get_prob_dropout_split(unlabeled_dataloader, device, unlabeled_features, examples)
    # deepAL+: pb = probs.mean(0)
    probs_mean = probs.mean(0)
    # deepAL+: entropy1 = (-pb*torch.log(pb)).sum(1)
    entropy1 = (-probs_mean*torch.log(probs_mean)).sum(1)
    # deepAL+: entropy2 = (-probs*torch.log(probs)).sum(2).mean(0)
    entropy2 = (-probs*torch.log(probs)).sum(2).mean(0)
    # deepAL+: uncertainties = entropy2 - entropy1
    uncertainties = entropy2 - entropy1
    # later on, we can use batch
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
    return unlabeled_idxs[uncertainties.sort()[1][:n]]

# main.py

### parameters

In [96]:
NUM_QUERY = args_input_batch
NUM_INIT_LB = args_input_initseed
NUM_ROUND = int(args_input_quota / args_input_batch)
DATA_NAME = args_input_dataset_name
STRATEGY_NAME = args_input_ALstrategy

### seed and device

In [97]:
SEED = 4666
# os.environ['TORCH_HOME']='./basicmodel'
os.environ["CUDA_VISIBLE_DEVICES"] = str(3)

# fix random seed
np.random.seed(SEED)
torch.manual_seed(SEED)
# torch.backends.cudnn.enabled  = True
# torch.backends.cudnn.benchmark= True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### start experiment

In [98]:
iteration = args_input_iteration
model_batch = args_input_model_batch
num_train_epochs = 3

all_acc = []
acq_time = []

# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = False

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

In [99]:
# repeate # iteration trials
while (iteration > 0): 
	iteration = iteration - 1

	## data, network, strategy
	model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased").to(device)
	optimizer = AdamW(model.parameters(), lr=1e-4)

	start = datetime.datetime.now()

	## generate initial labeled pool
	n_pool = len(train_dataset)
	labeled_idxs = np.zeros(n_pool, dtype=bool)

	tmp_idxs = np.arange(n_pool)
	np.random.shuffle(tmp_idxs)
	labeled_idxs[tmp_idxs[:NUM_INIT_LB]] = True

	run_0_labeled_idxs = np.arange(n_pool)[labeled_idxs]

	## record acc performance 
	acc = np.zeros(NUM_ROUND + 1) # quota/batch runs + run_0

	## load the selected train data to DataLoader
	train_dataloader = DataLoader(
		train_dataset.select(indices=run_0_labeled_idxs),
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)

	eval_dataloader = DataLoader(
		val_dataset, 
		collate_fn=default_data_collator, 
		batch_size=8
	)

	num_update_steps_per_epoch = len(train_dataloader)
	num_training_steps = num_train_epochs * num_update_steps_per_epoch

	lr_scheduler = get_scheduler(
		"linear",
		optimizer=optimizer,
		num_warmup_steps=0,
		num_training_steps=num_training_steps,
	)

	## print info
	print(DATA_NAME)
	print(STRATEGY_NAME)
	
	## round 0 accuracy
	to_train(num_train_epochs, train_dataloader, device, model, optimizer, lr_scheduler)

	acc[0] = get_pred(eval_dataloader, device, val_features, squad['validation'])['f1']

	# print('Round 0\ntesting accuracy {}'.format(acc[0]))
	# print('\n')
	
	# ## round 1 to rd
	# for rd in range(1, NUM_ROUND+1):
	# 	print('Round {}'.format(rd))

	# 	## query
	# 	if STRATEGY_NAME == 'RandomSampling':
	# 		q_idxs = random_sampling_query(labeled_idxs, NUM_QUERY)
	# 	elif STRATEGY_NAME == 'MarginSampling':
	# 		q_idxs = margin_sampling_query(n_pool, labeled_idxs, train_dataset, train_features, squad['train'], device, NUM_QUERY)
	# 	elif STRATEGY_NAME == 'LeastConfidence':
	# 		q_idxs = least_confidence_query(n_pool, labeled_idxs, train_dataset, train_features, squad['train'], device, NUM_QUERY)
	# 	elif STRATEGY_NAME == 'EntropySampling':
	# 		q_idxs = entropy_query(n_pool, labeled_idxs, train_dataset, train_features, squad['train'], device, NUM_QUERY)
	# 	elif STRATEGY_NAME == 'MarginSamplingDropout':
	# 		q_idxs = margin_sampling_dropout_query(n_pool, labeled_idxs, train_dataset, train_features, squad['train'], device, NUM_QUERY)
	# 	# elif STRATEGY_NAME == 'LeastConfidenceDropout':
	# 	# 	q_idxs = least_confidence_dropout_query(n_pool, labeled_idxs, train_dataset, train_features, squad['train'], device, NUM_QUERY)
	# 	# elif STRATEGY_NAME == 'EntropySamplingDropout':
	# 	# 	q_idxs = entropy_dropout_query(n_pool, labeled_idxs, train_dataset, train_features, squad['train'], device, NUM_QUERY)
	# 	# elif STRATEGY_NAME == 'VarRatio':
	# 	# 	q_idxs = var_ratio_query(n_pool, labeled_idxs, train_dataset, train_features, squad['train'], device, NUM_QUERY)
	# 	# elif STRATEGY_NAME == 'KMeansSampling':
	# 	# 	q_idxs = kmeans_query()
	# 	# elif STRATEGY_NAME == 'KCenterGreedy':
	# 	# 	q_idxs = kcenter_query()
	# 	# elif STRATEGY_NAME == 'KCenterGreedyPCA': # not sure
	# 	# 	q_idxs = 
	# 	# elif STRATEGY_NAME == 'BALDDropout':
	# 	# 	q_idxs = bayesian_query()
	# 	# elif STRATEGY_NAME == 'MeanSTD':
	# 	# 	q_idxs = mean_std_query()
	# 	# elif STRATEGY_NAME == 'BadgeSampling':
	# 	# 	q_idxs = badge_query()
	# 	# elif STRATEGY_NAME == 'LossPredictionLoss':
	# 	# 	# different net!
	# 	# 	q_idxs = loss_prediction_query()
	# 	# elif STRATEGY_NAME == 'CEALSampling':
	# 	# 	# why use 'CEALSampling' in STRATEGY_NAME
	# 	# 	q_idxs = ceal_query()
	# 	else:
	# 		raise NotImplementedError

	# 	## update
	# 	labeled_idxs[q_idxs] = True
	# 	run_rd_labeled_idxs = np.arange(n_pool)[labeled_idxs]

	# 	train_dataloader_rd = DataLoader(
	# 		train_dataset.select(indices=run_rd_labeled_idxs),
	# 		shuffle=True,
	# 		collate_fn=default_data_collator,
	# 		batch_size=8,
	# 	)

	# 	num_update_steps_per_epoch_rd = len(train_dataloader_rd)
	# 	num_training_steps_rd = num_train_epochs * num_update_steps_per_epoch_rd

	# 	lr_scheduler_rd = get_scheduler(
	# 		"linear",
	# 		optimizer=optimizer,
	# 		num_warmup_steps=0,
	# 		num_training_steps=num_training_steps_rd,
	# 	)

	# 	model_rd = AutoModelForQuestionAnswering.from_pretrained(model_dir).to(device)
	# 	optimizer_rd = AdamW(model_rd.parameters(), lr=1e-4)

	# 	## train
	# 	to_train(num_train_epochs, train_dataloader_rd, device, model_rd, optimizer_rd, lr_scheduler_rd)

	# 	## round rd accuracy
	# 	acc[rd] = get_pred(eval_dataloader, device, val_features, squad['validation'])['f1']
	# 	print('testing accuracy {}'.format(acc[rd]))
	# 	print('\n')

	# 	torch.cuda.empty_cache()
	
	# ## print results
	# print('SEED {}'.format(SEED))
	# print(STRATEGY_NAME)
	# print(acc)
	# all_acc.append(acc)
	
	# ## save model and record acq time
	# timestamp = re.sub('\.[0-9]*','_',str(datetime.datetime.now())).replace(" ", "_").replace("-", "").replace(":","")
	# final_model_dir = model_dir + '/' + timestamp + DATA_NAME+ '_'  + STRATEGY_NAME + '_' + str(NUM_QUERY) + '_' + str(NUM_INIT_LB) +  '_' + str(args_input.quota)
	# os.makedirs(final_model_dir, exist_ok=True)
	# end = datetime.datetime.now()
	# acq_time.append(round(float((end-start).seconds), 3))

	# final_model = AutoModelForQuestionAnswering.from_pretrained(model_dir).to(device)
	# model_to_save = final_model.module if hasattr(final_model, 'module') else final_model 
	# model_to_save.save_pretrained(final_model_dir)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_out

SQuAD
MarginSampling
num of train dataset 100


Training: 100%|██████████| 13/13 [00:06<00:00,  2.13it/s]


TRAIN done!


Evaluating_pred:   2%|▏         | 2/128 [00:00<00:26,  4.84it/s]

...

Evaluating_pred:  98%|█████████▊| 126/128 [00:24<00:00,  5.15it/s]

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 1.1323, -0.9586, -0.9191,  ..., -4.2439, -4.2478, -4.2461],
        [ 1.2705, -1.5363, -0.6011,  ..., -4.2328, -4.2434, -4.2388],
        [ 1.1742, -1.7398, -1.1542,  ..., -4.0995, -4.1205, -4.0873],
        ...,
        [ 1.0028, -1.5596, -0.4446,  ..., -4.2833, -4.3003, -4.3039],
        [ 1.5239, -1.6480, -1.3093,  ..., -4.1145, -4.1351, -4.1122],
        [ 1.6956, -1.6418, -0.4814,  ..., -4.2234, -4.2228, -4.2256]],
       device='cuda:0'), end_logits=tensor([[ 1.5999, -0.9607, -0.9553,  ..., -4.2421, -4.2386, -4.2279],
        [ 1.8144, -1.4704, -0.8407,  ..., -4.2671, -4.2594, -4.2549],
        [ 1.7088, -1.5029, -1.2593,  ..., -3.9498, -3.9562, -3.9363],
        ...,
        [ 1.3536, -1.6123,  0.1127,  ..., -4.1908, -4.1835, -4.1715],
        [ 2.0443, -1.5436, -0.7047,  ..., -4.0362, -4.0464, -4.0212],
        [ 2.2845, -1.7512, -0.5911,  ..., -4.1724, -4.1653, -4.1533]],
       device='cuda:0'), hidden_states=None

Evaluating_pred: 100%|██████████| 128/128 [00:24<00:00,  5.19it/s]

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 1.4357, -0.5832, -0.6467,  ..., -4.1105, -4.1121, -4.1112],
        [ 1.0775,  0.7633,  2.1925,  ..., -4.0069, -4.0067, -4.0189],
        [ 1.7481, -1.4053, -0.8086,  ..., -4.1238, -4.1344, -4.1181],
        ...,
        [ 0.9596, -1.7568, -1.5455,  ..., -4.3129, -4.3169, -4.3156],
        [ 1.0692, -1.8755, -1.0048,  ..., -4.2532, -4.2454, -4.2354],
        [ 1.5928, -1.4061, -1.3555,  ..., -4.1701, -4.1636, -4.1665]],
       device='cuda:0'), end_logits=tensor([[ 1.9839, -1.1459, -1.4283,  ..., -4.1348, -4.1290, -4.1275],
        [ 1.7188, -0.5440,  0.1459,  ..., -4.1969, -4.1896, -4.1823],
        [ 1.3852, -1.4305, -1.3032,  ..., -4.1314, -4.1245, -4.1068],
        ...,
        [ 1.7514, -1.8450, -2.1032,  ..., -4.2892, -4.2622, -4.2503],
        [ 1.8324, -1.6750, -1.4759,  ..., -4.0929, -4.0933, -4.0751],
        [ 2.2064, -1.3238, -1.7339,  ..., -4.1155, -4.1012, -4.1007]],
       device='cuda:0'), hidden_states=None

TypeError: unsupported format string passed to list.__format__

In [ ]:
all_acc

## query workspace

In [ ]:
# get unlable data
unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
len(unlabeled_idxs)

### test: query 5 data from 20 unlabeled_data

In [ ]:
# smaller data
unlabeled_idxs_20 = unlabeled_idxs[20:40]
unlabeled_data_20 = train_dataset.select(unlabeled_idxs_20)
unlabeled_feature_20 = train_features.select(unlabeled_idxs_20)
len(unlabeled_data_20)

In [ ]:
unlabeled_idxs_20

In [ ]:
unlabeled_dataloader = DataLoader(
		unlabeled_data_20,
		shuffle=True,
		collate_fn=default_data_collator,
		batch_size=8,
	)
len(unlabeled_dataloader.dataset)

In [ ]:
probs_list_dict_20 = get_prob(model, unlabeled_dataloader, device, unlabeled_feature_20, squad['train'])
# len(probs_list_dict_20)

In [ ]:
probs_list_dict_20

In [ ]:
uncertainties_dict = {}
for idx, probs in probs_list_dict_20.items():
    if len(probs) > 1: # if prob_dict['probs'] is not 0
        sort_probs = np.sort(probs)[::-1] # This method returns a copy of the array, leaving the original array unchanged.
        uncertainties = sort_probs[0] - sort_probs[1]
        uncertainties_dict[idx] = uncertainties
    elif idx:
        uncertainties_dict[idx] = np.array([0])
print('sort_probs:\n', sort_probs)
print('uncertainties_dict:\n', uncertainties_dict)
# deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]] 
sorted_uncertainties_list = sorted(uncertainties_dict.items(), key=lambda x: x[1], reverse=True)
    
unlabeled_idxs[[idx for (idx, uncertainties) in sorted_uncertainties_list[:5]]]

In [ ]:
probs_list_dict_20_dropout = get_prob_dropout(model, unlabeled_dataloader, device, unlabeled_data_20, squad['train'])

In [ ]:
for_check
# the prediction are the same

In [ ]:
probs_list_dict_20_dropout

In [ ]:
probs_list_dict_20

In [ ]:
confidence_list_dict = []
for d in probs_list_dict_20:
    if len(d['probs']) > 1: # if prob_dict['probs'] is not 0
        confidence = max(d['probs'])
        confidence_list_dict.append(
            {'idx': d['idx'], 
                'confidence': confidence}
                )
    elif d['idx']:
        confidence_list_dict.append(
            {'idx': d['idx'], 
                'confidence': np.array([0])}
                )
# deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
sorted_confidence_dict = sorted(confidence_list_dict, key=lambda d: d['confidence'])   
unlabeled_idxs_20[[confidence_dict['idx'][0] for confidence_dict in sorted_confidence_dict[:5]]]